In [2]:
import re

40. 係り受け解析結果の読み込み（形態素）

形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [6]:
class Morph:
    
    def __init__(self,surface,base,pos,pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def return_parsed(self):
        return [self.surface,self.base,self.pos,self.pos1]

pattern = r'^\* \d+ -?[0-9]+[A-Z] \d+/\d+ -?\d+\.\d+$'

with open('ai.ja.txt.parsed','r') as lines:
    for line in lines:
        matchs = re.findall(pattern,line)
        if line == 'EOS':
            break
        elif matchs:
            match_list = line.split(' ')
            if match_list[1] == '1':
                break
            print(match_list)
        else:
            line = line.split('\t')
            surface,sentences = line[0],line[1]
            sentence = sentences.split(',')
            (base,pos,pos1) = sentence[6],sentence[0],sentence[1]
            m = Morph(surface=surface,base=base,pos=pos,pos1=pos1)
            print(m.return_parsed())

['*', '0', '17D', '3/3', '0.388993\n']
['人工', '人工', '名詞', '一般']
['知能', '知能', '名詞', '一般']
['人工', '人工', '名詞', '一般']
['知能', '知能', '名詞', '一般']


41. 係り受け解析結果の読み込み（文節・係り受け）

40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [23]:
class Chunk:
    
    def __init__(self, morphs, dst, srcs, chunk_id):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs
        self.chunk_id = chunk_id
    
    def return_string_dependency(self):
        return "".join(self.morphs),self.dst

pattern = r'^\* \d+ -?[0-9]+[A-Z] \d+/\d+ -?\d+\.\d+$'
morphs = []
chunks = []
dst = None
srcs = None
chunk_id = -1

with open('ai.ja.txt.parsed','r') as lines:
    for line in lines:
        matchs = re.findall(pattern,line)
        if line == 'EOS':
            break
        elif matchs:
            match_list = line.split(' ')
            if chunk_id < 0:
                chunk_id = 0
            else:
                chunks.append(Chunk(morphs,dst,srcs,chunk_id))
            morphs = []
            dst = int(match_list[2].replace("D", ""))
            chunk_id = int(match_list[1])
            srcs=None
        else:
            line = line.split('\t')
            surface,sentences = line[0],line[1]
            sentence = sentences.split(',')
            (base,pos,pos1) = sentence[6],sentence[0],sentence[1]
            morphs.append(Morph(surface,base,pos,pos1).surface)

for chunk in chunks[:5]:
    (sentence,dst) = chunk.return_string_dependency()
    print(f"文節の文字列：{sentence}\n係り先のインデックス番号：{dst}\n")

文節の文字列：人工知能人工知能
係り先のインデックス番号：17

文節の文字列：（じんこうちのう、、
係り先のインデックス番号：17

文節の文字列：AI
係り先のインデックス番号：3

文節の文字列：〈エーアイ〉）とは、
係り先のインデックス番号：17

文節の文字列：「『計算
係り先のインデックス番号：5

